In [1]:
from pathlib import Path
import pandas as pd

BASE = Path("..").resolve()
RAW = BASE / "data" / "raw" / "fbref"

# какие сезоны для EPL уже должны быть
SEASONS = ["2019-2020","2020-2021","2021-2022","2022-2023","2023-2024","2024-2025"]
LEAGUE_CODE = "epl"

In [2]:
def read_csv_safe(p: Path) -> pd.DataFrame | None:
    if not p.exists():
        print(f"⚠️ not found: {p}")
        return None
    try:
        return pd.read_csv(p)
    except Exception as e:
        print(f"❌ read fail: {p.name} -> {e}")
        return None

In [3]:
rows, cols, samples = [], [], {}
for s in SEASONS:
    folder = RAW / f"{LEAGUE_CODE}_{s}"
    f = folder / "player_standard_stats.csv"
    df = read_csv_safe(f)
    if df is None:
        continue
    rows.append({"season": s, "rows": len(df), "cols": df.shape[1]})
    cols.append(tuple(df.columns))
    # сохраним по одному примеру для быстрой визуальной проверки
    samples[s] = df.head(3)

summary = pd.DataFrame(rows)
display(summary)

,season,rows,cols
0,2019-2020,542,37
1,2020-2021,553,37
2,2021-2022,567,37
3,2022-2023,591,37
4,2023-2024,603,37
5,2024-2025,596,37


In [4]:
unique_structs = {c for c in cols}
print(f"Уникальных наборов столбцов: {len(unique_structs)}")
if len(unique_structs) == 1:
    print("✅ Структура колонок одинакова во всех сезонах")
else:
    print("⚠️ Разные структуры колонок по сезонам — нужно сравнивать ниже")

# если есть расхождения — вывести различия
if len(unique_structs) > 1:
    structs = list(unique_structs)
    base = set(structs[0])
    for i, st in enumerate(structs[1:], 1):
        st_set = set(st)
        missing = base - st_set
        extra = st_set - base
        print(f"\nСравнение с шаблоном (вариант {i}):")
        if missing: print("  − отсутствуют:", sorted(missing))
        if extra:   print("  + лишние:", sorted(extra))

Уникальных наборов столбцов: 1
✅ Структура колонок одинакова во всех сезонах


In [5]:
last = SEASONS[-1]
df_last = read_csv_safe(RAW / f"{LEAGUE_CODE}_{last}" / "player_standard_stats.csv")
if df_last is not None:
    print(f"🔎 Проверка NaN для {last}:")
    na = df_last.isna().sum()
    na = na[na > 0].sort_values(ascending=False)
    display(na.head(20) if not na.empty else pd.Series(dtype=int, name="no_na"))
    
    print("\n📊 Типы данных (первые 20 колонок):")
    display(df_last.dtypes.head(20).to_frame("dtype").T)
    
    print("\n👀 Пример строк:")
    display(df_last.head(5))

🔎 Проверка NaN для 2024-2025:


nation    4
age       3
born      3
dtype: int64


📊 Типы данных (первые 20 колонок):


,rk,player,nation,pos,squad,age,born,mp,starts,min,90s,gls,ast,g_plus_a,g_pk,pk,pkatt,crdy,crdr,xg
dtype,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object



👀 Пример строк:


,rk,player,nation,pos,squad,age,born,mp,starts,min,...,ast_per90,g_plus_a_per90,g_pk_per90,g_plus_a_pk_per90,xg_per90,xag_per90,xg_xag_per90,npxg_per90,npxg_xag_per90,matches
0,1,Max Aarons,eng ENG,DF,Bournemouth,24,2000,3,1,86,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
1,2,Joshua Acheampong,eng ENG,DF,Chelsea,18,2006,4,2,170,...,0.00,0.00,0.00,0.00,0.12,0.00,0.12,0.12,0.12,Matches
2,3,Tyler Adams,us USA,MF,Bournemouth,25,1999,28,21,1965,...,0.14,0.14,0.00,0.14,0.07,0.05,0.12,0.07,0.12,Matches
3,4,Tosin Adarabioyo,eng ENG,DF,Chelsea,26,1997,22,15,1409,...,0.06,0.13,0.06,0.13,0.06,0.01,0.07,0.06,0.07,Matches
4,5,Simon Adingra,ci CIV,"FW,MF",Brighton,22,2002,29,12,1097,...,0.16,0.33,0.16,0.33,0.20,0.20,0.40,0.20,0.40,Matches


In [6]:
# Склеиваем player_standard_stats по всем сезонам EPL + базовая очистка

CATEGORICAL = {"player","nation","pos","squad","season"}

def load_players_one(season: str) -> pd.DataFrame | None:
    p = RAW / f"{LEAGUE_CODE}_{season}" / "player_standard_stats.csv"
    if not p.exists():
        print(f"⚠️ нет файла: {p}")
        return None
    df = pd.read_csv(p)

    # добавить сезон
    df["season"] = season

    # убрать служебную колонку, если попала
    if "matches" in df.columns:
        df = df.drop(columns=["matches"])

    # выбросить артефакты «шапок» внутри таблиц
    for col in ["player", "rk"]:
        if col in df.columns:
            df = df[df[col].astype(str).str.lower() != col]

    # привести числа из object → numeric (как получится)
    for c in df.columns:
        if c not in CATEGORICAL:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # косметика
    if "player" in df.columns:
        df["player"] = df["player"].astype(str).str.strip()
    if "squad" in df.columns:
        df["squad"] = df["squad"].astype(str).str.strip()

    return df.reset_index(drop=True)

players_all = pd.concat(
    [d for s in SEASONS if (d := load_players_one(s)) is not None],
    ignore_index=True
)

print(f"✅ players_all: {players_all.shape} (rows, cols)")
display(players_all.head(10))

✅ players_all: (3323, 37) (rows, cols)


,rk,player,nation,pos,squad,age,born,mp,starts,min,...,ast_per90,g_plus_a_per90,g_pk_per90,g_plus_a_pk_per90,xg_per90,xag_per90,xg_xag_per90,npxg_per90,npxg_xag_per90,season
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,29,29,2507,...,0.07,0.18,0.07,0.14,0.08,0.13,0.21,0.05,0.18,2019-2020
1,2,Max Aarons,eng ENG,DF,Norwich City,19.0,2000.0,36,36,3240,...,0.03,0.03,0.00,0.03,0.02,0.11,0.12,0.02,0.12,2019-2020
2,3,Tammy Abraham,eng ENG,FW,Chelsea,21.0,1997.0,34,25,2215,...,0.12,0.73,0.61,0.73,0.67,0.11,0.77,0.67,0.77,2019-2020
3,4,Che Adams,sct SCO,FW,Southampton,23.0,1996.0,30,12,1111,...,0.16,0.49,0.32,0.49,0.43,0.14,0.57,0.43,0.57,2019-2020
4,5,Adrián,es ESP,GK,Liverpool,32.0,1987.0,11,9,875,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2019-2020
5,6,Sergio Agüero,ar ARG,FW,Manchester City,31.0,1988.0,24,18,1452,...,0.19,1.18,0.87,1.05,0.90,0.14,1.04,0.75,0.89,2019-2020
6,7,Albian Ajeti,ch SUI,FW,West Ham,22.0,1997.0,9,0,139,...,0.00,0.00,0.00,0.00,0.11,0.05,0.16,0.11,0.16,2019-2020
7,8,Nathan Aké,nl NED,DF,Bournemouth,24.0,1995.0,29,29,2503,...,0.07,0.14,0.07,0.14,0.11,0.08,0.18,0.11,0.18,2019-2020
8,9,Marc Albrighton,eng ENG,"MF,DF",Leicester City,29.0,1989.0,20,9,867,...,0.31,0.31,0.00,0.31,0.04,0.22,0.27,0.04,0.27,2019-2020
9,10,Toby Alderweireld,be BEL,DF,Tottenham,30.0,1989.0,33,33,2957,...,0.06,0.12,0.06,0.12,0.02,0.04,0.06,0.02,0.06,2019-2020


In [7]:
# ВАЛИДАЦИЯ И СОХРАНЕНИЕ PLAYERS (EPL)

from pathlib import Path
import pandas as pd
import numpy as np

# на случай если не задано
try:
    LEAGUE_CODE
except NameError:
    LEAGUE_CODE = "epl"

try:
    BASE
except NameError:
    BASE = Path("..").resolve()

PROCESSED_PLAYERS = BASE / "data" / "processed" / "players"
PROCESSED_PLAYERS.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_PLAYERS / f"{LEAGUE_CODE}_players_all.csv"

# --- валидации ---
print(f"✅ Shape: {players_all.shape}")

# дубликаты по ключу (игрок в составе клуба в сезоне)
dup_mask = players_all.duplicated(subset=["season","player","squad"], keep=False)
dup_cnt = int(dup_mask.sum())
print(f"🔎 Дубликаты по (season, player, squad): {dup_cnt}")
if dup_cnt:
    display(players_all.loc[dup_mask, ["season","player","squad","pos","mp","starts","min"]].head(10))

# пропуски — топ-10 колонок
na = players_all.isna().sum()
na = na[na>0].sort_values(ascending=False)
if len(na):
    print("\n⚠️ Пропуски найдены (топ-10):")
    display(na.head(10))
else:
    print("\n✅ Пропусков нет")

# базовые диапазоны по числовым полям
num_cols = ["mp","starts","min","90s","gls","ast","xg","xag","npxg"]
present = [c for c in num_cols if c in players_all.columns]
desc = players_all[present].describe().T[["min","mean","max"]].round(2)
print("\n📊 Диапазоны ключевых числовых колонок:")
display(desc)

# --- сохранение ---
players_all.to_csv(out_path, index=False, encoding="utf-8")
print(f"\n📁 Сохранено в: {out_path}")

✅ Shape: (3323, 37)
🔎 Дубликаты по (season, player, squad): 0

⚠️ Пропуски найдены (топ-10):


nation    7
age       3
born      3
dtype: int64


📊 Диапазоны ключевых числовых колонок:


,min,mean,max
mp,1.0,19.80,38.0
starts,0.0,15.09,38.0
min,1.0,1356.07,3420.0
90s,0.0,15.07,38.0
gls,0.0,1.91,36.0
ast,0.0,1.36,20.0
xg,0.0,1.96,29.2
xag,0.0,1.40,20.0
npxg,0.0,1.82,23.0



📁 Сохранено в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/players/epl_players_all.csv


In [8]:
from pathlib import Path
import pandas as pd

BASE = Path("..").resolve()
RAW = BASE / "data" / "raw" / "fbref"
OUT = BASE / "data" / "processed" / "players"
OUT.mkdir(parents=True, exist_ok=True)

league_prefix = "laliga_"
files = sorted((p for p in RAW.glob(f"{league_prefix}*/player_standard_stats.csv")), key=lambda p: p.parent.name)

dfs = []
for f in files:
    season = f.parent.name.split("_", 1)[1]
    df = pd.read_csv(f)
    df["season"] = season
    dfs.append(df)

players_la = pd.concat(dfs, ignore_index=True)

print(f"✅ La Liga players: {players_la.shape} (rows, cols)")
dups = players_la.duplicated(subset=["season","player","squad"]).sum()
print(f"🔎 Дубликаты по (season, player, squad): {dups}")

na = players_la.isna().sum()
na_top = na[na > 0].sort_values(ascending=False).head(10)
if len(na_top):
    print("\n⚠️ Пропуски (топ-10):")
    display(na_top)
else:
    print("\n✅ Пропусков нет в топ-колонках")

num_cols = ["mp","starts","min","90s","gls","ast","xg","xag","npxg"]
rng = players_la[num_cols].agg(["min","mean","max"]).round(2).T
print("\n📊 Диапазоны ключевых числовых колонок:")
display(rng)

out_path = OUT / "laliga_players_all.csv"
players_la.to_csv(out_path, index=False)
print(f"📁 Сохранено в: {out_path}")

✅ La Liga players: (3715, 38) (rows, cols)
🔎 Дубликаты по (season, player, squad): 134

✅ Пропусков нет в топ-колонках


TypeError: Could not convert string '11423313227361327641718232107132813151113534MP62441414123493714320926253619116933530812MP1730122853733212333301614112483128351523133MP334112181715271723792036351935151124311335352MP301316424373241231712632433124823196341822MP37272365521636303423515368311222251833181830MP20233112633610111735110124291713322527336434MP2020341617122017371452923173632830123543082MP425122173337131819261011411352835242121456MP2116172995211930525143435329372338224203429MP35343736178335291751927112322729243535311510MP831331422283122429291732526323611743262524MP3813617335272981513121512136613312053534MP34242521311625233622933297153432313427291153418MP31359361263115118236184233892263551572034MP381334352352624233029137271351353332132026MP2102226342613220735363013163528111932036MP1110191526329232312477353253136256281549MP30333528272613230371481191162519212571943125MP9172885251129219221232635143335342421314613MP332813871130262811761418273622218142034283436MP1429251335332213611113

In [9]:
# --- FIX: привести числовые колонки к float и пересчитать диапазоны ---

import re
num_cols = ["mp","starts","min","90s","gls","ast","xg","xag","npxg"]

# оставить только существующие из списка (на всякий случай)
num_cols = [c for c in num_cols if c in players_la.columns]

def to_num(s):
    # в строках иногда встречаются символы вроде 'MP', 'Matches', пробелы и т.п.
    # вычищаем всё, кроме цифр, точки и минуса, после чего преобразуем к числу
    return pd.to_numeric(
        pd.Series(s, dtype="string")
          .str.replace(r"[^\d\.\-]", "", regex=True)
          .replace({"": None}),
        errors="coerce"
    )

for c in num_cols:
    players_la[c] = to_num(players_la[c])

# пересчёт диапазонов
rng = players_la[num_cols].agg(["min","mean","max"]).round(2).T
print("📊 Диапазоны ключевых числовых колонок (после очистки):")
display(rng)

📊 Диапазоны ключевых числовых колонок (после очистки):


,min,mean,max
mp,1.0,19.58,38.0
starts,0.0,14.03,38.0
min,1.0,1259.30,3420.0
90s,0.0,16.86,90.0
gls,0.0,1.58,31.0
ast,0.0,1.10,21.0
xg,0.0,1.64,27.1
xag,0.0,1.15,15.4
npxg,0.0,1.47,24.0


In [10]:
# --- Удаляем дубликаты и сохраняем финальный датасет ---
before = len(players_la)
players_la = players_la.drop_duplicates(subset=["season", "player", "squad"], keep="first")
after = len(players_la)
print(f"✅ Убрано дубликатов: {before - after}, финальный размер: {after}")

out_path = OUT / "laliga_players_all.csv"
players_la.to_csv(out_path, index=False)
print(f"📁 Финальный файл сохранён в: {out_path}")

✅ Убрано дубликатов: 134, финальный размер: 3581
📁 Финальный файл сохранён в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/players/laliga_players_all.csv
